# Interfacing Python with ROS (ROS 1)

This Jupyter notebook presents essential interfacing between Python and the Robot Operating System (ROS). The sections are based on [ROS Wiki's ROS Tutorial](https://wiki.ros.org/ROS/Tutorials), however, only ROS messages and basic command line tools are discussed in the context of using them with Jupyter Notebooks.

**If you need a more detailed breakdown of a certain ROS feature, please refer to the [full ROS Tutorial](https://wiki.ros.org/ROS/Tutorials)**.

In order to execute the cells presented in this notebook, ROS and a [Jupyter notebook interface](https://jupyter.org/) must be installed alongside each other. Refer to [this repo](https://github.com/SimonSchwaiger/ros-ml-container) if you have trouble setting that up yourself.

# Table of Concents
* [**ROS Command Line Tools**](#roscmd)
* [**ROS Python interfacing**](#interfacing)
    - [**Rospy import and ROS node initialisation**](#rospyinit)
    - [**Handling of ROS data types**](#datatypehandling)
    - [**Basic publisher**](#publisher)
    - [**Basic subscriber**](#subscriber)
* [**Essential ROS message types**](#messagetypes)

## ROS Command Line Tools <a class="anchor" id="roscmd"></a>

In order to test our interface with ROS, ROS command line tools will be used to introspect ROS nodes and send/receive messages. The following command line tools are required for this notebook

* **`roscore`**: starts ROS 1. In ROS 1, roscore must be active in the background if you want to use any ROS features.

* **`rosnode`**: manages and inspects ROS nodes. Nodes are individual processes in ROS.
   - `rosnode list`: Lists all currently active nodes
   - `rosnode info /node_name`: Displays information about a specific node

* **`rostopic`**: interacts with ROS topics. Topics are message-passing channels in ROS.
   - `rostopic list`: Lists all available topics
   - `rostopic echo /topic_name`: Prints the messages being published on a topic
   - `rostopic pub /topic_name [msg_type] [args]`: Publishes a message on a topic from the command line

### ROS command line hello world

1. Open a terminal and start ROS using the `roscore` command
2. Open a second terminal and check running nodes using `rosnode list` and active topics using `rostopic list`. Since no ROS nodes have been started yet, nothing apart from rosout should be shown.
3. Monitor the `/hello_world` topic by using `rostopic echo /hello_world`.
4. Open another terminal and use `rostopic pub /hello_world std_msgs/String "data: 'Hello from terminal 3'" -1` to send a message to the second terminal through ROS. `std_msgs/String` denotes the ROS datatype, while `"data: 'Hello from terminal 3!'"` sets the message and `-1` causes the message to be sent only once.
5. Stop all terminals using `Ctrl+C` in each terminal

These command line tools will help us test the functionality of the Python code examples used throughout this notebook.

## ROS Python interfacing <a class="anchor" id="interfacing"></a>

### Rospy import and ROS node initialisation <a class="anchor" id="rospyinit"></a>

To enable any interaction in ROS, the `rospy` module needs to be imported and initialised. The node name is provided as an argument to the initialisation function. Since ROS is natively compatible with Python, a Jupyter notebook can interact with ROS as a standard ROS node. Once initialized, the node is active and can publish/subscribe to topics, provide/consume services, and interact with the ROS ecosystem. We can only initialise one node per notebook, since all notebook cells share a common Python kernel. Furthermore, each node name in a ROS system must be unique.

In [1]:
import rospy

rospy.init_node("my_first_robot_node")

To test the code, first make sure that a terminal with `roscore` is running in the background. Now, run the code cell above. You should now see your node when checking the output of `rosnode list` in another terminal.

_**Note:** The notebook's Python kernel stays active in the background, even if no code cells are currently running. If you restart your roscore after having initialised a node, you need to restart your notebook's Python kernel and reinitialise your node in order to connect to the new roscore. This is done in JupyterLab by clicking on "Kernel" in the top bar and then selecting "Restart Kernel..."._

### Handling of ROS data types <a class="anchor" id="datatypehandling"></a>

ROS defines its own message types tailored to various robotics tasks. These data structures are different from Python's message types (Python String $\neq$ ROS String). They are organised in packages and need to be imported individually. For the remainder of this tutorial, we will use ROS' String message from the std_msgs (= standard messages) package. Documentation this datatype [(link)](https://docs.ros.org/en/noetic/api/std_msgs/html/msg/String.html) tells us, that ROS String messages consist of just one field called *data*, which can store text.

In [2]:
# Import the message. Replace std_msgs.msg if a message is part of a different package such as geometry_msgs.
from std_msgs.msg import String

# Instantiate empty message
msg = String()
# Fill message with data
msg.data = "Hello, ROS!"

print(msg)

data: "Hello, ROS!"


Alternatively, the data field can be directly populated when instantiating the message using keyword arguments. The accepted keyword arguments depend on what fields a ROS message type contains.

In [3]:
msg2 = String(data="Hello, ROS! (but short)")

print(msg2)

data: "Hello, ROS! (but short)"


### Basic publisher <a class="anchor" id="publisher"></a>

A publisher is a ROS node that sends data to a topic. After node initialisation, a publisher object can be instantiated. When creating the publisher, a topic, message type and queue_size arguments have to be provided. Topic is is the name of the used ROS topic, message type the used ROS datatype and queue_size denotes how many messages are buffered when sending messages faster than ROS can process. You need a separate publsher for each topic you want to send data to and the first registered publisher on a new topic determines the topic's message type. A publisher instance can be reused an unlimited number of times to send new messages to a topic.

In [4]:
from std_msgs.msg import String

# Define message
msg = String(data="Hello, ROS!")

# Create publisher
pub = rospy.Publisher('/greetings', String, queue_size=1)

# Send message
pub.publish(msg)

To test the code, make sure that a roscore is running and that a node has been registered in this notebook's kernel by running the *init_node* method once. Afterwards, open a second terminal and use the `rostopic echo /greetings` command to monitor the topic this code example sends a message to. Finally, execute the code cell.

### Basic subscriber <a class="anchor" id="subscriber"></a>

A subscriber is a ROS node that listens to data from a topic. Setup is similar to the publisher, however, a callback function needs to be provided. This function is executed in the background every time a message is received on the subscribed topic. In this example, we use a very simple class to store the latest received message.

In [27]:
from std_msgs.msg import String

class store_latest_msg:
    def __init__(self):
        # Connect the subscriber to the topic and callback
        self.sub = rospy.Subscriber('/greetings', String, self.callback)
        self.msg = None

    # Define callback that stores the latest mesage
    def callback(self, msg):
        self.msg = msg.data

sub = store_latest_msg()

To test this code, make sure that a roscore is running and that a node has been registered in this notebook's kernel. Afterwards, run the code cell. You can then send messages to the */greetings* topic by executing the example code in *Basic publisher* or running `rostopic pub /greetings std_msgs/String "data: 'Hello from the terminal'" -1`.

Finally, the stored message can be printed.

In [28]:
print(sub.msg)

Hello from the terminal


## Essential ROS message types <a class="anchor" id="messagetypes"></a>

This list provides a brief overview of ROS message types required for basic mobile robot applications. Please refer to the ROS message documentation [(link)](https://docs.ros.org/en/noetic/api/) for a detailed breakdown of fields included in these message types.

* **geometry_msgs/Twist [(documentation link)](https://docs.ros.org/en/melodic/api/geometry_msgs/html/msg/Twist.html)**

    Represents velocity in free space (linear and angular). When working with non-holonomic 2D mobile robots, linear velocity in x (forward-backward movement) and angular velocity in z (steering) are the only two values of interest.

    ```python
    from geometry_msgs.msg import Twist
   
    twist = Twist()
    twist.linear.x = 0.5  # Move forward at 0.5 m/s
    twist.angular.z = 0.1 # Rotate at 0.1 rad/s
     
    ```

* **sensor_msgs/LaserScan [(documentation link)](https://docs.ros.org/en/noetic/api/sensor_msgs/html/msg/LaserScan.html)**

    Contains data from a laser range-finder, used in navigation and obstacle detection.

    ```python
    from sensor_msgs.msg import LaserScan

    scan = LaserScan()
    scan.ranges    # List of measured distances
    scan.angle_min # Starting angle of measurement
    scan.angle_max # Last angle of measurement
     
   ```

* **nav_msgs/Odometry [(documentation link)](https://docs.ros.org/en/noetic/api/nav_msgs/html/msg/Odometry.html)**

    Provides robot odometry information, which includes position and velocity. This is one of the more complex datatypes. Along the robot pose it also includes a header denoting the measurement's reference coordinate system (frame_id) and timestamp as well as covariance matrices for the pose denoting uncertainty in the measurements.

    ```python
    from nav_msgs.msg import Odometry

    odom = Odometry()
    odom.header.frame_id # Reference coordinate system/frame
    odom.heater.stamp    # Timestamp of measurement
    odom.pose.pose       # Pose

    ```
